<img src='images/Home.png' width='100'>

<img src='images/Homes.png' width='600'>

## Feature Engineering of House Prices Dataset
<b>Goal</b>: Predict sales prices and practice feature engineering, RFs, and gradient boosting

#### Kaggle Dataset Link
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [210]:
# import all neccesary libraries
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="white")
# Display all columns of the dataset
pd.pandas.set_option('display.max_columns',None)

### Feature Engineering on Training Data

In [211]:
df = pd.read_csv('train.csv')
df

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0            Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1            Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2            Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3            Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4            Lvl    AllPub       FR2       Gtl      NoRidge       Norm   
...          ...       ...       ...       ...          ...        ...   
1455         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
1456         Lvl    AllPub    Inside       Gtl       NWAmes       Norm   
1457         Lvl    AllPub    Inside       Gtl      Crawfor       Norm   
1458         Lvl    AllPub    Inside       Gtl        NAmes       Norm   
1459         Lvl    AllPub    Inside       Gtl      Edwards       Norm   

     Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0          Norm     1Fam     2Story            7            5       2003   
1          Norm     1Fam     1Story            6            8       1976   
2          Norm     1Fam     2Story            7            5       2001   
3          Norm     1Fam     2Story            7            5       1915   
4          Norm     1Fam     2Story            8            5       2000   
...         ...      ...        ...          ...          ...        ...   
1455       Norm     1Fam     2Story            6            5       1999   
1456       Norm     1Fam     1Story            6            6       1978   
1457       Norm     1Fam     2Story            7            9       1941   
1458       Norm     1Fam     1Story            5            6       1950   
1459       Norm     1Fam     1Story            5            6       1965   

      YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0             2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1             1976     Gable  CompShg     MetalSd     MetalSd       None   
2             2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3             1970     Gable  CompShg     Wd Sdng     Wd Shng       None   
4             2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   
...            ...       ...      ...         ...         ...        ...   
1455          2000     Gable  CompShg     VinylSd     VinylSd       None   
1456          1988     Gable  CompShg     Plywood     Plywood      Stone   
1457          2006     Gable  CompShg     CemntBd     CmentBd       None   
1458          1996       Hip  CompShg     MetalSd     MetalSd       None   
1459          1965     Gable  CompShg     HdBoard     HdBoard       None   

      MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond  \
0          196.0        Gd        TA      PConc       Gd       TA   
1            0.0        TA        TA     CBlock       Gd       TA   
2          162.0        Gd        TA      PConc       Gd       TA   
3            0.0        TA        TA     BrkTil       TA       Gd   

In [212]:
# To stop any data leakage always split the data and then apply feature Engineering
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df['SalePrice'], test_size=0.1, random_state=0)

In [213]:
X_train.shape, X_test.shape

((1314, 81), (146, 81))

### Missing Values

#### Categorical features

In [214]:
# Handling missing values of Categorical features

# Find null values for all categorical features
cat_null_values = [feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes == 'O']

# Count percentage of null values for each feature
for feature in cat_null_values:
        print("{}: {}% missing values".format(feature,np.round(df[feature].isnull().mean(),2)))

Alley: 0.94% missing values
MasVnrType: 0.01% missing values
BsmtQual: 0.03% missing values
BsmtCond: 0.03% missing values
BsmtExposure: 0.03% missing values
BsmtFinType1: 0.03% missing values
BsmtFinType2: 0.03% missing values
FireplaceQu: 0.47% missing values
GarageType: 0.06% missing values
GarageFinish: 0.06% missing values
GarageQual: 0.06% missing values
GarageCond: 0.06% missing values
PoolQC: 1.0% missing values
Fence: 0.81% missing values
MiscFeature: 0.96% missing values


In [215]:
# Fill up all missing values into null categorical features
def fill_cat_features(dataset, features_null):
    data = dataset.copy()
    data[features_null] = data[features_null].fillna('Missing')
    return data

df = fill_cat_features(df, cat_null_values)
df[cat_null_values].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [217]:
df.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street    Alley LotShape  \
0   1          60       RL         65.0     8450   Pave  Missing      Reg   
1   2          20       RL         80.0     9600   Pave  Missing      Reg   
2   3          60       RL         68.0    11250   Pave  Missing      IR1   
3   4          70       RL         60.0     9550   Pave  Missing      IR1   
4   5          60       RL         84.0    14260   Pave  Missing      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story            7            5       2003   
1       Norm     1Fam     1Story            6            8       1976   
2       Norm     1Fam     2Story            7            5       2001   
3       Norm     1Fam     2Story            7            5       1915   
4       Norm     1Fam     2Story            8            5       2000   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0          2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1          1976     Gable  CompShg     MetalSd     MetalSd       None   
2          2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3          1970     Gable  CompShg     Wd Sdng     Wd Shng       None   
4          2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0       196.0        Gd        TA      PConc       Gd       TA           No   
1         0.0        TA        TA     CBlock       Gd       TA           Gd   
2       162.0        Gd        TA      PConc       Gd       TA           Mn   
3         0.0        TA        TA     BrkTil       TA       Gd           No   
4       350.0        Gd        TA      PConc       Gd       TA           Av   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          GLQ         706          Unf           0        150          856   
1          ALQ         978          Unf           0        284         1262   
2          GLQ         486          Unf           0        434          920   
3          ALQ         216          Unf           0        540          756   
4          GLQ         655          Unf           0        490         1145   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        Ex          Y      SBrkr       856       854             0   
1    GasA        Ex          Y      SBrkr      1262         0             0   
2    GasA        Ex          Y      SBrkr       920       866             0   
3    GasA        Gd          Y      SBrkr       961       756             0   
4    GasA        Ex          Y      SBrkr      1145      1053             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0       1710             1             0         2         1             3   
1       1262             0             1         2         0             3   
2       1786             1             0         2         1             3   
3       1717             1             0         1         0             3   
4       2198             1             0         2         1             4   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          Gd             8        Typ           0     Missing   
1             1          TA             6        Typ           1          TA   
2             1          Gd             6        Typ           1          TA   
3             1      

#### Numerical features

In [218]:
# Handling missing values of Numerical features

# Find null values for all Numerical features
num_null_values = [feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes != 'O']

# Count percentage of null values for each feature
for feature in num_null_values:
        print("{}: {}% missing values".format(feature,np.round(df[feature].isnull().mean(),2)))

LotFrontage: 0.18% missing values
MasVnrArea: 0.01% missing values
GarageYrBlt: 0.06% missing values


In [219]:
# Replace null values of numerical features
for feature in num_null_values:
    # Replacing null values with median because there are lost of outliners.
    median_value = df[feature].median()
    df[feature].fillna(median_value, inplace=True)
df[num_null_values].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

#### Year Features

In [220]:
# Temporal variables (Date-Time features)
for feature in ['YearBuilt','YearRemodAdd','GarageYrBlt']:       
    df[feature]=df['YrSold'] - df[feature]

In [221]:
df[['YearBuilt','YearRemodAdd','GarageYrBlt']].head()

YearBuilt  YearRemodAdd  GarageYrBlt
0          5             5          5.0
1         31            31         31.0
2          7             6          7.0
3         91            36          8.0
4          8             8          8.0

### Numerical Features

In [222]:
df.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street    Alley LotShape  \
0   1          60       RL         65.0     8450   Pave  Missing      Reg   
1   2          20       RL         80.0     9600   Pave  Missing      Reg   
2   3          60       RL         68.0    11250   Pave  Missing      IR1   
3   4          70       RL         60.0     9550   Pave  Missing      IR1   
4   5          60       RL         84.0    14260   Pave  Missing      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story            7            5          5   
1       Norm     1Fam     1Story            6            8         31   
2       Norm     1Fam     2Story            7            5          7   
3       Norm     1Fam     2Story            7            5         91   
4       Norm     1Fam     2Story            8            5          8   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0             5     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1            31     Gable  CompShg     MetalSd     MetalSd       None   
2             6     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3            36     Gable  CompShg     Wd Sdng     Wd Shng       None   
4             8     Gable  CompShg     VinylSd     VinylSd    BrkFace   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0       196.0        Gd        TA      PConc       Gd       TA           No   
1         0.0        TA        TA     CBlock       Gd       TA           Gd   
2       162.0        Gd        TA      PConc       Gd       TA           Mn   
3         0.0        TA        TA     BrkTil       TA       Gd           No   
4       350.0        Gd        TA      PConc       Gd       TA           Av   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          GLQ         706          Unf           0        150          856   
1          ALQ         978          Unf           0        284         1262   
2          GLQ         486          Unf           0        434          920   
3          ALQ         216          Unf           0        540          756   
4          GLQ         655          Unf           0        490         1145   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        Ex          Y      SBrkr       856       854             0   
1    GasA        Ex          Y      SBrkr      1262         0             0   
2    GasA        Ex          Y      SBrkr       920       866             0   
3    GasA        Gd          Y      SBrkr       961       756             0   
4    GasA        Ex          Y      SBrkr      1145      1053             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0       1710             1             0         2         1             3   
1       1262             0             1         2         0             3   
2       1786             1             0         2         1             3   
3       1717             1             0         1         0             3   
4       2198             1             0         2         1             4   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          Gd             8        Typ           0     Missing   
1             1          TA             6        Typ           1          TA   
2             1          Gd             6        Typ           1          TA   
3             1      

In [223]:
target_feature = df['SalePrice']

In [224]:
# tranform numerical skewed values to log normal distribution
import numpy as np

num_features = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']

for feature in num_features:
    df[feature] = np.log(df[feature])

In [225]:
df.head()

Id  MSSubClass MSZoning  LotFrontage   LotArea Street    Alley LotShape  \
0   1          60       RL     4.174387  9.041922   Pave  Missing      Reg   
1   2          20       RL     4.382027  9.169518   Pave  Missing      Reg   
2   3          60       RL     4.219508  9.328123   Pave  Missing      IR1   
3   4          70       RL     4.094345  9.164296   Pave  Missing      IR1   
4   5          60       RL     4.430817  9.565214   Pave  Missing      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story            7            5          5   
1       Norm     1Fam     1Story            6            8         31   
2       Norm     1Fam     2Story            7            5          7   
3       Norm     1Fam     2Story            7            5         91   
4       Norm     1Fam     2Story            8            5          8   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0             5     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1            31     Gable  CompShg     MetalSd     MetalSd       None   
2             6     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3            36     Gable  CompShg     Wd Sdng     Wd Shng       None   
4             8     Gable  CompShg     VinylSd     VinylSd    BrkFace   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0       196.0        Gd        TA      PConc       Gd       TA           No   
1         0.0        TA        TA     CBlock       Gd       TA           Gd   
2       162.0        Gd        TA      PConc       Gd       TA           Mn   
3         0.0        TA        TA     BrkTil       TA       Gd           No   
4       350.0        Gd        TA      PConc       Gd       TA           Av   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          GLQ         706          Unf           0        150          856   
1          ALQ         978          Unf           0        284         1262   
2          GLQ         486          Unf           0        434          920   
3          ALQ         216          Unf           0        540          756   
4          GLQ         655          Unf           0        490         1145   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        Ex          Y      SBrkr  6.752270       854             0   
1    GasA        Ex          Y      SBrkr  7.140453         0             0   
2    GasA        Ex          Y      SBrkr  6.824374       866             0   
3    GasA        Gd          Y      SBrkr  6.867974       756             0   
4    GasA        Ex          Y      SBrkr  7.043160      1053             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0   7.444249             1             0         2         1             3   
1   7.140453             0             1         2         0             3   
2   7.487734             1             0         2         1             3   
3   7.448334             1             0         1         0             3   
4   7.695303             1             0         2         1             4   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          Gd             8        Typ           0     Missing   
1             1          TA             6        Typ           1          TA   
2             1          Gd             6        Typ           1          TA   
3             1

### Handling rare categorical features

In [227]:
# get all categorical features 
cat_features = [feature for feature in df.columns if df[feature].dtypes == 'O']
# replace rare category of feature to 'Rare_var'
for feature in cat_features:
    temp = df.groupby(feature)['SalePrice'].count()/len(df)
    temp_df = temp[temp>0.01].index
    df[feature] = np.where(df[feature].isin(temp_df),df[feature],'Rare_var')

In [228]:
df.head(50)

Id  MSSubClass  MSZoning  LotFrontage   LotArea Street    Alley LotShape  \
0    1          60        RL     4.174387  9.041922   Pave  Missing      Reg   
1    2          20        RL     4.382027  9.169518   Pave  Missing      Reg   
2    3          60        RL     4.219508  9.328123   Pave  Missing      IR1   
3    4          70        RL     4.094345  9.164296   Pave  Missing      IR1   
4    5          60        RL     4.430817  9.565214   Pave  Missing      IR1   
5    6          50        RL     4.442651  9.554993   Pave  Missing      IR1   
6    7          20        RL     4.317488  9.218705   Pave  Missing      Reg   
7    8          60        RL     4.234107  9.247829   Pave  Missing      IR1   
8    9          50        RM     3.931826  8.719317   Pave  Missing      Reg   
9   10         190        RL     3.912023  8.911934   Pave  Missing      Reg   
10  11          20        RL     4.248495  9.323669   Pave  Missing      Reg   
11  12          60        RL     4.442651  9.386308   Pave  Missing      IR1   
12  13          20        RL     4.234107  9.470240   Pave  Missing      IR2   
13  14          20        RL     4.510860  9.273503   Pave  Missing      IR1   
14  15          20        RL     4.234107  9.298351   Pave  Missing      IR1   
15  16          45        RM     3.931826  8.719317   Pave  Missing      Reg   
16  17          20        RL     4.234107  9.327323   Pave  Missing      IR1   
17  18          90        RL     4.276666  9.286468   Pave  Missing      Reg   
18  19          20        RL     4.189655  9.524786   Pave  Missing      Reg   
19  20          20        RL     4.248495  8.930626   Pave  Missing      Reg   
20  21          60        RL     4.615121  9.562053   Pave  Missing      IR1   
21  22          45        RM     4.043051  8.915835   Pave     Grvl      Reg   
22  23          20        RL     4.317488  9.184202   Pave  Missing      Reg   
23  24         120        RM     3.784190  8.348538   Pave  Missing      Reg   
24  25          20        RL     4.234107  9.017484   Pave  Missing      IR1   
25  26          20        RL     4.700480  9.563108   Pave  Missing      Reg   
26  27          20        RL     4.094345  8.881836   Pave  Missing      Reg   
27  28          20        RL     4.584967  9.348187   Pave  Missing      Reg   
28  29          20        RL     3.850148  9.700208   Pave  Missing      IR1   
29  30          30        RM     4.094345  8.752107   Pave  Missing      IR1   
30  31          70  Rare_var     3.912023  9.047821   Pave     Pave      Reg   
31  32          20        RL     4.234107  9.052985   Pave  Missing      IR1   
32  33          20        RL     4.442651  9.310095   Pave  Missing      Reg   
33  34          20        RL     4.248495  9.264071   Pave  Missing      IR1   
34  35         120        RL     4.094345  8.897409   Pave  Missing      Reg   
35  36          60        RL     4.682131  9.504352   Pave  Missing      Reg   
36  37          20        RL     4.718499  9.292750   Pave  Missing      Reg   
37  38          20        RL     4.304065  9.051579   Pave  Missing      Reg   
38  39          20        RL     4.219508  8.977399   Pave  Missing      Reg   
39  40          90        RL     4.174387  8.706159   Pave  Missing      Reg   
40  41          20        RL     4.430817  9.066239   Pave  Missing      Reg   
41  42          20        RL     4.744932  9.735365   Pave  Missing      Reg   
42  43          85        RL     4.234107  9.124782   Pave  Missing      IR1   
43  44          20        RL     4.234107  9.126959   Pave  Missing      IR1   
44  45          20        RL     4.248495  8.980298   Pave  Missing      Reg   
45  46         120        RL     4.110874  8.943506   Pave  Missing      Reg   
46  47          50        RL     3.871201  9.458918   Pave  Missing      IR1   
47  48          20        FV     4.430817  9.314340   Pave  Missing      Reg   
48  49         190        RM     3.496508  8.402007   Pave  Missing      Reg   
49  

In [229]:
df['MSZoning'].groupby(df['MSZoning']).count()

MSZoning
FV            65
RH            16
RL          1151
RM           218
Rare_var      10
Name: MSZoning, dtype: int64

In [230]:
# provide lables to each categorial features
for feature in cat_features:
    labels_ordered=df.groupby([feature])['SalePrice'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    df[feature]=df[feature].map(labels_ordered)

In [231]:
df.head()

Id  MSSubClass  MSZoning  LotFrontage   LotArea  Street  Alley  LotShape  \
0   1          60         3     4.174387  9.041922       1      2         0   
1   2          20         3     4.382027  9.169518       1      2         0   
2   3          60         3     4.219508  9.328123       1      2         1   
3   4          70         3     4.094345  9.164296       1      2         1   
4   5          60         3     4.430817  9.565214       1      2         1   

   LandContour  Utilities  LotConfig  LandSlope  Neighborhood  Condition1  \
0            1          1          0          0            14           2   
1            1          1          2          0            11           1   
2            1          1          0          0            14           2   
3            1          1          1          0            16           2   
4            1          1          2          0            22           2   

   Condition2  BldgType  HouseStyle  OverallQual  OverallCond  YearBuilt  \
0           1         3           5            7            5          5   
1           1         3           3            6            8         31   
2           1         3           5            7            5          7   
3           1         3           5            7            5         91   
4           1         3           5            8            5          8   

   YearRemodAdd  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  \
0             5          0         0           10           10           2   
1            31          0         0            4            3           1   
2             6          0         0           10           10           2   
3            36          0         0            2            4           1   
4             8          0         0           10           10           2   

   MasVnrArea  ExterQual  ExterCond  Foundation  BsmtQual  BsmtCond  \
0       196.0          2          3           4         3         3   
1         0.0          1          3           2         3         3   
2       162.0          2          3           4         3         3   
3         0.0          1          3           1         2         4   
4       350.0          2          3           4         3         3   

   BsmtExposure  BsmtFinType1  BsmtFinSF1  BsmtFinType2  BsmtFinSF2  \
0             1             6         706             5           0   
1             4             4         978             5           0   
2             2             6         486             5           0   
3             1             4         216             5           0   
4             3             6         655             5           0   

   BsmtUnfSF  TotalBsmtSF  Heating  HeatingQC  CentralAir  Electrical  \
0        150          856        2          4           1           3   
1        284         1262        2          4           1           3   
2        434          920        2          4           1           3   
3        540          756        2          3           1           3   
4        490         1145        2          4           1           3   

   1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  \
0  6.752270       854             0   7.444249             1             0   
1  7.140453         0             0   7.140453             0             1   
2  6.824374       866             0   7.487734             1             0   
3  6.867974       756             0   7.448334             1             0   
4  7.043160      1053             0   7.695303             1             0   

   FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  KitchenQual  TotRmsAbvGrd  \
0         2         1             3             1            2             8   
1         2         0             3             1            1             6   
2         2         1             3             1            2             6   
3         1         0             3             1        

### Feature Scaling

In [232]:
# scaling all features of the dataset
from sklearn.preprocessing import MinMaxScaler
feature_scale = [feature for feature in df.columns if feature not in ['Id','SalePrice']]
scaler = MinMaxScaler()
scaler.fit(df[feature_scale])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [233]:
scaler.transform(df[feature_scale])

array([[0.23529412, 0.75      , 0.41820812, ..., 0.5       , 0.66666667,
        0.75      ],
       [0.        , 0.75      , 0.49506375, ..., 0.25      , 0.66666667,
        0.75      ],
       [0.23529412, 0.75      , 0.434909  , ..., 0.5       , 0.66666667,
        0.75      ],
       ...,
       [0.29411765, 0.75      , 0.42385922, ..., 1.        , 0.66666667,
        0.75      ],
       [0.        , 0.75      , 0.434909  , ..., 1.        , 0.66666667,
        0.75      ],
       [0.        , 0.75      , 0.47117546, ..., 0.5       , 0.66666667,
        0.75      ]])

In [234]:
# concating the 'Id' and 'Saleprice' with scaled data
df = pd.concat([df[['Id', 'SalePrice']].reset_index(drop=True), pd.DataFrame(scaler.transform(df[feature_scale]), 
                columns=feature_scale)], axis=1)

In [235]:
df.to_csv('train_new.csv', index=False)
target_feature.to_csv('target.csv', index=False)

/Users/suroor/Documents/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


### Feature Engineering on Test Data

In [236]:
df_test = pd.read_csv('test.csv')
df_test.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0  1461          20       RH         80.0    11622   Pave   NaN      Reg   
1  1462          20       RL         81.0    14267   Pave   NaN      IR1   
2  1463          60       RL         74.0    13830   Pave   NaN      IR1   
3  1464          60       RL         78.0     9978   Pave   NaN      IR1   
4  1465         120       RL         43.0     5005   Pave   NaN      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl        NAmes      Feedr   
1         Lvl    AllPub    Corner       Gtl        NAmes       Norm   
2         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
3         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
4         HLS    AllPub    Inside       Gtl      StoneBr       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     1Story            5            6       1961   
1       Norm     1Fam     1Story            6            6       1958   
2       Norm     1Fam     2Story            5            5       1997   
3       Norm     1Fam     2Story            6            6       1998   
4       Norm   TwnhsE     1Story            8            5       1992   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0          1961     Gable  CompShg     VinylSd     VinylSd       None   
1          1958       Hip  CompShg     Wd Sdng     Wd Sdng    BrkFace   
2          1998     Gable  CompShg     VinylSd     VinylSd       None   
3          1998     Gable  CompShg     VinylSd     VinylSd    BrkFace   
4          1992     Gable  CompShg     HdBoard     HdBoard       None   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0         0.0        TA        TA     CBlock       TA       TA           No   
1       108.0        TA        TA     CBlock       TA       TA           No   
2         0.0        TA        TA      PConc       Gd       TA           No   
3        20.0        TA        TA      PConc       TA       TA           No   
4         0.0        Gd        TA      PConc       Gd       TA           No   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          Rec       468.0          LwQ       144.0      270.0        882.0   
1          ALQ       923.0          Unf         0.0      406.0       1329.0   
2          GLQ       791.0          Unf         0.0      137.0        928.0   
3          GLQ       602.0          Unf         0.0      324.0        926.0   
4          ALQ       263.0          Unf         0.0     1017.0       1280.0   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        TA          Y      SBrkr       896         0             0   
1    GasA        TA          Y      SBrkr      1329         0             0   
2    GasA        Gd          Y      SBrkr       928       701             0   
3    GasA        Ex          Y      SBrkr       926       678             0   
4    GasA        Ex          Y      SBrkr      1280         0             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0        896           0.0           0.0         1         0             2   
1       1329           0.0           0.0         1         1             3   
2       1629           0.0           0.0         2         1             3   
3       1604           0.0           0.0         2         1             3   
4       1280           0.0           0.0         2         0             2   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          TA             5        Typ           0         NaN   
1             1          Gd             6        Typ           0         NaN   
2             1          TA             6        Typ           1          TA   
3             1          Gd  

In [237]:
df_test.shape

(1459, 80)

### Missing Values

#### Categorical Features

In [238]:
# Handling missing values of Categorical features

# Find null values for all categorical features
categorical_null_values = [feature for feature in df_test.columns if df_test[feature].isnull().sum()>1 
                           and df_test[feature].dtypes == 'O']

# Count percentage of null values for each feature
for feature in categorical_null_values:
        print("{}: {}% missing values".format(feature,np.round(df_test[feature].isnull().mean(),2)))

MSZoning: 0.0% missing values
Alley: 0.93% missing values
Utilities: 0.0% missing values
MasVnrType: 0.01% missing values
BsmtQual: 0.03% missing values
BsmtCond: 0.03% missing values
BsmtExposure: 0.03% missing values
BsmtFinType1: 0.03% missing values
BsmtFinType2: 0.03% missing values
Functional: 0.0% missing values
FireplaceQu: 0.5% missing values
GarageType: 0.05% missing values
GarageFinish: 0.05% missing values
GarageQual: 0.05% missing values
GarageCond: 0.05% missing values
PoolQC: 1.0% missing values
Fence: 0.8% missing values
MiscFeature: 0.97% missing values


In [239]:
# Fill up all missing values into null categorical features
def fill_cat_features_test(dataset, features_null):
    data = dataset.copy()
    data[features_null] = data[features_null].fillna('Missing')
    return data

df_test = fill_cat_features_test(df_test, cat_null_values)
df_test[cat_null_values].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [240]:
df_test.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street    Alley LotShape  \
0  1461          20       RH         80.0    11622   Pave  Missing      Reg   
1  1462          20       RL         81.0    14267   Pave  Missing      IR1   
2  1463          60       RL         74.0    13830   Pave  Missing      IR1   
3  1464          60       RL         78.0     9978   Pave  Missing      IR1   
4  1465         120       RL         43.0     5005   Pave  Missing      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl        NAmes      Feedr   
1         Lvl    AllPub    Corner       Gtl        NAmes       Norm   
2         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
3         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
4         HLS    AllPub    Inside       Gtl      StoneBr       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     1Story            5            6       1961   
1       Norm     1Fam     1Story            6            6       1958   
2       Norm     1Fam     2Story            5            5       1997   
3       Norm     1Fam     2Story            6            6       1998   
4       Norm   TwnhsE     1Story            8            5       1992   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0          1961     Gable  CompShg     VinylSd     VinylSd       None   
1          1958       Hip  CompShg     Wd Sdng     Wd Sdng    BrkFace   
2          1998     Gable  CompShg     VinylSd     VinylSd       None   
3          1998     Gable  CompShg     VinylSd     VinylSd    BrkFace   
4          1992     Gable  CompShg     HdBoard     HdBoard       None   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0         0.0        TA        TA     CBlock       TA       TA           No   
1       108.0        TA        TA     CBlock       TA       TA           No   
2         0.0        TA        TA      PConc       Gd       TA           No   
3        20.0        TA        TA      PConc       TA       TA           No   
4         0.0        Gd        TA      PConc       Gd       TA           No   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          Rec       468.0          LwQ       144.0      270.0        882.0   
1          ALQ       923.0          Unf         0.0      406.0       1329.0   
2          GLQ       791.0          Unf         0.0      137.0        928.0   
3          GLQ       602.0          Unf         0.0      324.0        926.0   
4          ALQ       263.0          Unf         0.0     1017.0       1280.0   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        TA          Y      SBrkr       896         0             0   
1    GasA        TA          Y      SBrkr      1329         0             0   
2    GasA        Gd          Y      SBrkr       928       701             0   
3    GasA        Ex          Y      SBrkr       926       678             0   
4    GasA        Ex          Y      SBrkr      1280         0             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0        896           0.0           0.0         1         0             2   
1       1329           0.0           0.0         1         1             3   
2       1629           0.0           0.0         2         1             3   
3       1604           0.0           0.0         2         1             3   
4       1280           0.0           0.0         2         0             2   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          TA             5        Typ           0     Missing   
1             1          Gd             6        Typ           0     Missing   
2             1          TA             6        Typ           1          TA   
3          

#### Numerical features

In [241]:
# Handling missing values of Numerical features

# Find null values for all Numerical features
numerical_null_values = [feature for feature in df_test.columns if df_test[feature].isnull().sum()>1 
                         and df_test[feature].dtypes != 'O']

# Count percentage of null values for each feature
for feature in numerical_null_values:
        print("{}: {}% missing values".format(feature,np.round(df_test[feature].isnull().mean(),2)))

LotFrontage: 0.16% missing values
MasVnrArea: 0.01% missing values
BsmtFullBath: 0.0% missing values
BsmtHalfBath: 0.0% missing values
GarageYrBlt: 0.05% missing values


In [242]:
# Replace null values of numerical features
for feature in numerical_null_values:
    # Replacing null values with median because there are lost of outliners.
    median_value = df_test[feature].median()
    df_test[feature].fillna(median_value, inplace=True)
df_test[numerical_null_values].isnull().sum()

LotFrontage     0
MasVnrArea      0
BsmtFullBath    0
BsmtHalfBath    0
GarageYrBlt     0
dtype: int64

#### Year Features

In [243]:
# Temporal variables (Date-Time features)
for feature in ['YearBuilt','YearRemodAdd','GarageYrBlt']:       
    df_test[feature]=df_test['YrSold'] - df_test[feature]

In [244]:
df[['YearBuilt','YearRemodAdd','GarageYrBlt']].head()

YearBuilt  YearRemodAdd  GarageYrBlt
0   0.036765      0.098361     0.046729
1   0.227941      0.524590     0.289720
2   0.051471      0.114754     0.065421
3   0.669118      0.606557     0.074766
4   0.058824      0.147541     0.074766

### Numerical Features

In [245]:
df_test.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street    Alley LotShape  \
0  1461          20       RH         80.0    11622   Pave  Missing      Reg   
1  1462          20       RL         81.0    14267   Pave  Missing      IR1   
2  1463          60       RL         74.0    13830   Pave  Missing      IR1   
3  1464          60       RL         78.0     9978   Pave  Missing      IR1   
4  1465         120       RL         43.0     5005   Pave  Missing      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl        NAmes      Feedr   
1         Lvl    AllPub    Corner       Gtl        NAmes       Norm   
2         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
3         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
4         HLS    AllPub    Inside       Gtl      StoneBr       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     1Story            5            6         49   
1       Norm     1Fam     1Story            6            6         52   
2       Norm     1Fam     2Story            5            5         13   
3       Norm     1Fam     2Story            6            6         12   
4       Norm   TwnhsE     1Story            8            5         18   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0            49     Gable  CompShg     VinylSd     VinylSd       None   
1            52       Hip  CompShg     Wd Sdng     Wd Sdng    BrkFace   
2            12     Gable  CompShg     VinylSd     VinylSd       None   
3            12     Gable  CompShg     VinylSd     VinylSd    BrkFace   
4            18     Gable  CompShg     HdBoard     HdBoard       None   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0         0.0        TA        TA     CBlock       TA       TA           No   
1       108.0        TA        TA     CBlock       TA       TA           No   
2         0.0        TA        TA      PConc       Gd       TA           No   
3        20.0        TA        TA      PConc       TA       TA           No   
4         0.0        Gd        TA      PConc       Gd       TA           No   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          Rec       468.0          LwQ       144.0      270.0        882.0   
1          ALQ       923.0          Unf         0.0      406.0       1329.0   
2          GLQ       791.0          Unf         0.0      137.0        928.0   
3          GLQ       602.0          Unf         0.0      324.0        926.0   
4          ALQ       263.0          Unf         0.0     1017.0       1280.0   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        TA          Y      SBrkr       896         0             0   
1    GasA        TA          Y      SBrkr      1329         0             0   
2    GasA        Gd          Y      SBrkr       928       701             0   
3    GasA        Ex          Y      SBrkr       926       678             0   
4    GasA        Ex          Y      SBrkr      1280         0             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0        896           0.0           0.0         1         0             2   
1       1329           0.0           0.0         1         1             3   
2       1629           0.0           0.0         2         1             3   
3       1604           0.0           0.0         2         1             3   
4       1280           0.0           0.0         2         0             2   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          TA             5        Typ           0     Missing   
1             1          Gd             6        Typ           0     Missing   
2             1          TA             6        Typ           1          TA   
3          

In [246]:
# tranform numerical skewed values to log normal distribution
import numpy as np

numerical_features = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea']

for feature in numerical_features:
    df_test[feature] = np.log(df_test[feature])

In [247]:
df_test.head()

Id  MSSubClass MSZoning  LotFrontage   LotArea Street    Alley LotShape  \
0  1461          20       RH     4.382027  9.360655   Pave  Missing      Reg   
1  1462          20       RL     4.394449  9.565704   Pave  Missing      IR1   
2  1463          60       RL     4.304065  9.534595   Pave  Missing      IR1   
3  1464          60       RL     4.356709  9.208138   Pave  Missing      IR1   
4  1465         120       RL     3.761200  8.518193   Pave  Missing      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl        NAmes      Feedr   
1         Lvl    AllPub    Corner       Gtl        NAmes       Norm   
2         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
3         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
4         HLS    AllPub    Inside       Gtl      StoneBr       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     1Story            5            6         49   
1       Norm     1Fam     1Story            6            6         52   
2       Norm     1Fam     2Story            5            5         13   
3       Norm     1Fam     2Story            6            6         12   
4       Norm   TwnhsE     1Story            8            5         18   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0            49     Gable  CompShg     VinylSd     VinylSd       None   
1            52       Hip  CompShg     Wd Sdng     Wd Sdng    BrkFace   
2            12     Gable  CompShg     VinylSd     VinylSd       None   
3            12     Gable  CompShg     VinylSd     VinylSd    BrkFace   
4            18     Gable  CompShg     HdBoard     HdBoard       None   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0         0.0        TA        TA     CBlock       TA       TA           No   
1       108.0        TA        TA     CBlock       TA       TA           No   
2         0.0        TA        TA      PConc       Gd       TA           No   
3        20.0        TA        TA      PConc       TA       TA           No   
4         0.0        Gd        TA      PConc       Gd       TA           No   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          Rec       468.0          LwQ       144.0      270.0        882.0   
1          ALQ       923.0          Unf         0.0      406.0       1329.0   
2          GLQ       791.0          Unf         0.0      137.0        928.0   
3          GLQ       602.0          Unf         0.0      324.0        926.0   
4          ALQ       263.0          Unf         0.0     1017.0       1280.0   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        TA          Y      SBrkr  6.797940         0             0   
1    GasA        TA          Y      SBrkr  7.192182         0             0   
2    GasA        Gd          Y      SBrkr  6.833032       701             0   
3    GasA        Ex          Y      SBrkr  6.830874       678             0   
4    GasA        Ex          Y      SBrkr  7.154615         0             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0   6.797940           0.0           0.0         1         0             2   
1   7.192182           0.0           0.0         1         1             3   
2   7.395722           0.0           0.0         2         1             3   
3   7.380256           0.0           0.0         2         1             3   
4   7.154615           0.0           0.0         2         0             2   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          TA             5        Typ           0     Missing   
1             1          Gd             6        Typ           0     Missing   
2             1          TA             6        Typ           1          TA   
3    

In [248]:
df_test['MSZoning'].groupby(df_test['MSZoning']).count()

MSZoning
C (all)      15
FV           74
RH           10
RL         1114
RM          242
Name: MSZoning, dtype: int64

### Handling rare categorical features

In [249]:
# get all categorical features 
categorical_features = [feature for feature in df_test.columns if df_test[feature].dtypes == 'O']
# replace rare category of feature to 'Rare_var'
for feature in categorical_features:
    temp = df_test.groupby(feature).count()/len(df_test)
    temp_df = temp[temp>0.01].index
    df_test[feature] = np.where(df_test[feature].isin(temp_df),df_test[feature],'Rare_var')

In [250]:
df_test['MSZoning'].groupby(df_test['MSZoning']).count()

MSZoning
C (all)       15
FV            74
RH            10
RL          1114
RM           242
Rare_var       4
Name: MSZoning, dtype: int64

In [251]:
# provide lables to each categorial features
for feature in categorical_features:
    labels_ordered = df_test.groupby([feature])['LotFrontage'].mean().sort_values().index
    labels_ordered = {k:i for i,k in enumerate(labels_ordered,0)}
    df_test[feature] = df_test[feature].map(labels_ordered)

In [252]:
df_test.head(5)

Id  MSSubClass  MSZoning  LotFrontage   LotArea  Street  Alley  LotShape  \
0  1461          20         0     4.382027  9.360655       0      2         1   
1  1462          20         4     4.394449  9.565704       0      2         2   
2  1463          60         4     4.304065  9.534595       0      2         2   
3  1464          60         4     4.356709  9.208138       0      2         2   
4  1465         120         4     3.761200  8.518193       0      2         2   

   LandContour  Utilities  LotConfig  LandSlope  Neighborhood  Condition1  \
0            0          0          2          0            17           6   
1            0          0          4          0            17           3   
2            0          0          2          0            13           3   
3            0          0          2          0            13           3   
4            2          0          2          0             6           3   

   Condition2  BldgType  HouseStyle  OverallQual  OverallCond  YearBuilt  \
0           0         4           5            5            6         49   
1           0         4           5            6            6         52   
2           0         4           0            5            5         13   
3           0         4           0            6            6         12   
4           0         1           5            8            5         18   

   YearRemodAdd  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  \
0            49          1         0            9           10           0   
1            52          4         0            5            9           1   
2            12          1         0            9           10           0   
3            12          1         0            9           10           1   
4            18          1         0            7            8           0   

   MasVnrArea  ExterQual  ExterCond  Foundation  BsmtQual  BsmtCond  \
0         0.0          0          1           3         2         1   
1       108.0          0          1           3         2         1   
2         0.0          0          1           4         1         1   
3        20.0          0          1           4         2         1   
4         0.0          1          1           4         1         1   

   BsmtExposure  BsmtFinType1  BsmtFinSF1  BsmtFinType2  BsmtFinSF2  \
0             0             2       468.0             2       144.0   
1             0             1       923.0             1         0.0   
2             0             4       791.0             1         0.0   
3             0             4       602.0             1         0.0   
4             0             1       263.0             1         0.0   

   BsmtUnfSF  TotalBsmtSF  Heating  HeatingQC  CentralAir  Electrical  \
0      270.0        882.0        0          1           1           3   
1      406.0       1329.0        0          1           1           3   
2      137.0        928.0        0          0           1           3   
3      324.0        926.0        0          2           1           3   
4     1017.0       1280.0        0          2           1           3   

   1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  \
0  6.797940         0             0   6.797940           0.0           0.0   
1  7.192182         0             0   7.192182           0.0           0.0   
2  6.833032       701             0   7.395722           0.0           0.0   
3  6.830874       678             0   7.380256           0.0           0.0   
4  7.154615         0             0   7.154615           0.0           0.0   

   FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  KitchenQual  TotRmsAbvGrd  \
0         1         0             2             1            0             5   
1         1         1             3             1            2             6   
2         2         1             3             1            0             6   
3         2         1             3            

### Feature Scaling

In [253]:
# scaling all features of the dataset
from sklearn.preprocessing import MinMaxScaler
feature_scaler = [feature for feature in df_test.columns if feature not in ['Id']]
scale = MinMaxScaler()
scale.fit(df_test[feature_scaler])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [254]:
scale.transform(df_test[feature_scaler])

array([[0.        , 0.        , 0.59344538, ..., 1.        , 0.44444444,
        0.2       ],
       [0.        , 0.8       , 0.59895721, ..., 1.        , 0.44444444,
        0.2       ],
       [0.23529412, 0.8       , 0.55885415, ..., 1.        , 0.44444444,
        0.2       ],
       ...,
       [0.        , 0.8       , 0.90099208, ..., 0.        , 0.44444444,
        0.4       ],
       [0.38235294, 0.8       , 0.48035069, ..., 0.        , 0.44444444,
        0.2       ],
       [0.23529412, 0.8       , 0.55885415, ..., 0.        , 0.44444444,
        0.2       ]])

In [255]:
# concating the 'Id' with scaled data
df_test = pd.concat([df_test[['Id']].reset_index(drop=True), pd.DataFrame(scaler.transform(df_test[feature_scale]), 
                columns=feature_scale)], axis=1)

In [256]:
df_test.head()

Id  MSSubClass  MSZoning  LotFrontage   LotArea  Street  Alley  LotShape  \
0  1461    0.000000       0.0     0.495064  0.428726     0.0    1.0  0.333333   
1  1462    0.000000       1.0     0.499662  0.468857     0.0    1.0  0.666667   
2  1463    0.235294       1.0     0.466207  0.462769     0.0    1.0  0.666667   
3  1464    0.235294       1.0     0.485693  0.398875     0.0    1.0  0.666667   
4  1465    0.588235       1.0     0.265271  0.263841     0.0    1.0  0.666667   

   LandContour  Utilities  LotConfig  LandSlope  Neighborhood  Condition1  \
0     0.000000        0.0        0.5        0.0      0.772727         1.2   
1     0.000000        0.0        1.0        0.0      0.772727         0.6   
2     0.000000        0.0        0.5        0.0      0.590909         0.6   
3     0.000000        0.0        0.5        0.0      0.590909         0.6   
4     0.666667        0.0        0.5        0.0      0.272727         0.6   

   Condition2  BldgType  HouseStyle  OverallQual  OverallCond  YearBuilt  \
0         0.0      1.00         1.0     0.444444        0.625   0.360294   
1         0.0      1.00         1.0     0.555556        0.625   0.382353   
2         0.0      1.00         0.0     0.444444        0.500   0.095588   
3         0.0      1.00         0.0     0.555556        0.625   0.088235   
4         0.0      0.25         1.0     0.777778        0.500   0.132353   

   YearRemodAdd  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  \
0      0.819672        0.5       0.0          0.9          1.0        0.00   
1      0.868852        2.0       0.0          0.5          0.9        0.25   
2      0.213115        0.5       0.0          0.9          1.0        0.00   
3      0.213115        0.5       0.0          0.9          1.0        0.25   
4      0.311475        0.5       0.0          0.7          0.8        0.00   

   MasVnrArea  ExterQual  ExterCond  Foundation  BsmtQual  BsmtCond  \
0      0.0000   0.000000   0.333333        0.75      0.50      0.25   
1      0.0675   0.000000   0.333333        0.75      0.50      0.25   
2      0.0000   0.000000   0.333333        1.00      0.25      0.25   
3      0.0125   0.000000   0.333333        1.00      0.50      0.25   
4      0.0000   0.333333   0.333333        1.00      0.25      0.25   

   BsmtExposure  BsmtFinType1  BsmtFinSF1  BsmtFinType2  BsmtFinSF2  \
0           0.0      0.333333    0.082920      0.333333    0.097693   
1           0.0      0.166667    0.163536      0.166667    0.000000   
2           0.0      0.666667    0.140149      0.166667    0.000000   
3           0.0      0.666667    0.106662      0.166667    0.000000   
4           0.0      0.166667    0.046598      0.166667    0.000000   

   BsmtUnfSF  TotalBsmtSF  Heating  HeatingQC  CentralAir  Electrical  \
0   0.115582     0.144354      0.0       0.25         1.0         1.0   
1   0.173801     0.217512      0.0       0.25         1.0         1.0   
2   0.058647     0.151882      0.0       0.00         1.0         1.0   
3   0.138699     0.151555      0.0       0.50         1.0         1.0   
4   0.435360     0.209493      0.0       0.50         1.0         1.0   

   1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  \
0  0.373438  0.000000           0.0   0.349081           0.0           0.0   
1  0.522632  0.000000           0.0   0.488544           0.0           0.0   
2  0.386718  0.339467           0.0   0.560546           0.0           0.0   
3  0.385901  0.328329           0.0   0.555075           0.0           0.0   
4  0.508416  0.000000           0.0   0.475254           0.0           0.0   

   FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  KitchenQual  TotRmsAbvGrd  \
0  0.333333       0.0         0.250      0.333333     0.000000      0.250000   
1  0.333333       0.5         0.375      0.333333     0.666667      0.333333   
2  0.666667       0.5         0.375      0.333333     0.000000      0.333333   
3  0.666667       0.5         0.375      0.3333

In [257]:
df_test.to_csv('test_new.csv', index=False)